In [1]:
from tagger.core.mongo.models.order import Order
import pandas as pd
import simplejson
from datetime import datetime


In [2]:
def get_data(d_from, d_to):
    orders = Order.objects(orderstatus__nin=["CREATED", "RECREATED", "PAYMENT_PENDING"], user__mobile__nin=["01028861089", "01097711882", "01077591771"], created__gte=d_from, created__lt=d_to)
    print(d_from, d_to, [x for x in orders][0].id, [x for x in orders][-1].id)
    cancel_orders = [x for x in orders if "CANCEL" in x.orderstatus]
    timedeal_orders = orders.filter(ordertype="TIMEDEAL_ORDER")
    normal_orders = orders.filter(ordertype="NORMAL_ORDER")
#     df = pd.DataFrame.from_dict([x.to_mongo() for x in orders])
    count = len(orders)
    cancel_count = len(cancel_orders)
    cancel_rate = len(cancel_orders) / len(orders) if len(orders) != 0 else None
    avg_price = sum([x.totalprice for x in orders]) / len(orders) if len(orders) != 0 else None
    nominal_total = sum([x.totalprice for x in orders])
    non_cancel_total = sum([x.totalprice for x in orders if "CANCEL" not in x.orderstatus])

    timedeal_count = len(timedeal_orders)
    timedeal_cancel_count = len([x for x in timedeal_orders if "CANCEL" in x.orderstatus])
    timedeal_cancel_rate = len([x for x in timedeal_orders if "CANCEL" in x.orderstatus])  / len(timedeal_orders) if len(timedeal_orders) != 0 else None
    timedeal_avg_price = sum([x.totalprice for x in timedeal_orders]) / len(timedeal_orders) if len(timedeal_orders) != 0 else None

    normal_count = len(normal_orders)
    normal_cancel_count = len([x for x in normal_orders if "CANCEL" in x.orderstatus])
    normal_cancel_rate = len([x for x in normal_orders if "CANCEL" in x.orderstatus])  / len(normal_orders) if len(normal_orders) != 0 else None
    normal_avg_price = sum([x.totalprice for x in normal_orders]) / len(normal_orders) if len(normal_orders) != 0 else None
    normal_order_avg_price = sum([x.totalprice for x in normal_orders]) / sum([len(x.orders) for x in normal_orders]) if sum([len(x.orders) for x in normal_orders]) != 0 else None
    return {
        "count": count,
        "cancel_count": cancel_count,
        "cancel_rate": cancel_rate,
        "avg_price": avg_price,
        "nominal_total": nominal_total,        
        "non_cancel_total": non_cancel_total,        
        "timedeal_count": timedeal_count,
        "timedeal_cancel_count": timedeal_cancel_count,
        "timedeal_cancel_rate": timedeal_cancel_rate,
        "timedeal_avg_price": timedeal_avg_price,
        "normal_count": normal_count,
        "normal_cancel_count": normal_cancel_count,
        "normal_cancel_rate": normal_cancel_rate,
        "normal_avg_price": normal_avg_price,
        "normal_order_avg_price": normal_order_avg_price,
    }
    


In [3]:
def get_row(d_from, d_to):
    data = get_data(d_from, d_to)
    return {"week": d_from, **data}

In [7]:
rows = [
    get_row(datetime(2021,9,4,15), datetime(2021,9,11,15)),
    get_row(datetime(2021,9,11,15), datetime(2021,9,18,15)),
    get_row(datetime(2021,9,18,15), datetime(2021,9,25,15)),
    get_row(datetime(2021,9,25,15), datetime(2021,10,2,15)),
    get_row(datetime(2021,10,2,15), datetime(2021,10,9,15)),
    get_row(datetime(2021,10,9,15), datetime(2021,10,16,15)),
    get_row(datetime(2021,10,16,15), datetime(2021,10,23,15)),
    get_row(datetime(2021,10,23,15), datetime(2021,10,30,15)),
]

2021-09-04 15:00:00 2021-09-11 15:00:00 61356adc77d38e5e9c24c1c2 613c48ed77d38e5e9c24c5a1
2021-09-11 15:00:00 2021-09-18 15:00:00 613d8b6677d38e5e9c24c5ca 61456913b0244b79594ee833
2021-09-18 15:00:00 2021-09-25 15:00:00 61461c924b20da9d98ecee8e 614f1b1c4b20da9d98ecf5f1
2021-09-25 15:00:00 2021-10-02 15:00:00 614fe6b54b20da9d98ecf679 61582e9c4b20da9d98ecfd1d
2021-10-02 15:00:00 2021-10-09 15:00:00 615a74493e2ace567810b8df 6161ad62a789fd943b7cdf8d
2021-10-09 15:00:00 2021-10-16 15:00:00 6161ca2aa789fd943b7cdfbf 616ac84717fc2e5a1f9e8b7d
2021-10-16 15:00:00 2021-10-23 15:00:00 616afa8917fc2e5a1f9e8ba0 61740dea17fc2e5a1f9e9f1c
2021-10-23 15:00:00 2021-10-30 15:00:00 61742b8317fc2e5a1f9e9f59 617b4fed17fc2e5a1f9ead08


In [8]:
pd.DataFrame(rows).to_clipboard()

In [9]:
pd.DataFrame(rows)

,week,count,cancel_count,cancel_rate,avg_price,nominal_total,non_cancel_total,timedeal_count,timedeal_cancel_count,timedeal_cancel_rate,timedeal_avg_price,normal_count,normal_cancel_count,normal_cancel_rate,normal_avg_price,normal_order_avg_price
0,2021-09-04 15:00:00,7,4,0.571429,133582.857143,935080,414000,4,2,0.500000,195645.000000,3,2,0.666667,50833.333333,50833.333333
1,2021-09-11 15:00:00,19,8,0.421053,120971.578947,2298460,1462240,15,6,0.400000,123774.000000,4,2,0.500000,110462.500000,55231.250000
2,2021-09-18 15:00:00,35,18,0.514286,111095.142857,3888330,2027890,25,13,0.520000,107890.000000,10,5,0.500000,119108.000000,85077.142857
3,2021-09-25 15:00:00,31,15,0.483871,109676.451613,3399970,2219185,18,7,0.388889,86235.555556,13,8,0.615385,142133.076923,115483.125000
4,2021-10-02 15:00:00,27,13,0.481481,280179.814815,7564855,3717080,19,8,0.421053,309216.578947,8,5,0.625000,211217.500000,187748.888889
5,2021-10-09 15:00:00,51,19,0.372549,311738.039216,15898640,11406320,38,14,0.368421,367815.263158,13,5,0.384615,147820.000000,106758.888889
6,2021-10-16 15:00:00,137,59,0.430657,214128.613139,29335620,17142650,118,49,0.415254,211269.406780,19,10,0.526316,231885.789474,157351.071429
7,2021-10-23 15:00:00,82,20,0.243902,288965.365854,23695160,14888560,66,14,0.212121,295285.757576,16,6,0.375000,262893.750000,200300.000000


In [90]:
all_orders = Order.objects(orderstatus__nin=["CREATED", "RECREATED", "PAYMENT_PENDING"], user__mobile__nin=["01028861089", "01097711882", "01077591771"])

In [91]:
timedeal_orders = all_orders.filter(ordertype="TIMEDEAL_ORDER")

In [93]:
td_items = [x.orders[0].alloffproduct for x in timedeal_orders]

In [95]:
brands = {}
for item in td_items:
    if item.brand.keyname in brands:
        brands[item.brand.keyname] += [item]
    else:
        brands[item.brand.keyname] = [item]        

In [107]:
brand_rows = []
for keyname in brands.keys():
    items = brands[keyname]
    count = len(items)
    price_sum = sum([x.discountedprice for x in items])
    brand_rows += [{"brand": keyname, "count": count, "total": price_sum, "avg_price": int(price_sum/count)}]

In [108]:
pd.DataFrame(brand_rows)

,brand,count,total,avg_price
0,MINE,14,2584330,184595
1,THURSDAYISLAND,1,59800,59800
2,SYSTEM,32,3730100,116565
3,SJSJ,15,1415820,94388
4,TIME,8,1106550,138318
5,DEWL,7,819870,117124
6,CCCOLLECT,6,622400,103733
7,LATT,2,139120,69560
8,OBZEE,8,1942590,242823
9,O2ND,1,72380,72380


In [8]:
orders = Order.objects(orderstatus__nin=["CREATED", "RECREATED", "PAYMENT_PENDING"], user__mobile__nin=["01028861089", "01097711882", "01077591771"])

In [32]:
usermap = {}
for order in orders:
    existing = usermap[order.user._id] if order.user._id in usermap else []
    usermap[order.user._id] = existing + [order]

In [33]:
usermap_data = []
for key, val in usermap.items():
    usermap_data.append({"count": len(val), "amountsum": sum([x.totalprice for x in val]), "userid": key})

In [34]:
pd.DataFrame(usermap_data).to_clipboard()

In [54]:
for o in [x for x in Order.objects(orderstatus__nin=["CREATED", "RECREATED", "PAYMENT_PENDING"], user__mobile__nin=["01028861089", "01097711882", "01077591771"], created__gte=datetime(2021,10,16,15), created__lt=datetime(2021,10,23,15))]:
    print(o.totalprice, o.id)

97350 616afa8917fc2e5a1f9e8ba0
161100 616b1f5b17fc2e5a1f9e8bb7
292500 616b8a1717fc2e5a1f9e8bed
97350 616b8a2517fc2e5a1f9e8bee
129000 616bcbcc17fc2e5a1f9e8c69
129000 616bcc2c17fc2e5a1f9e8c6e
129000 616bcccd17fc2e5a1f9e8c79
97350 616bcdd517fc2e5a1f9e8c82
236000 616bcdf117fc2e5a1f9e8c84
236000 616bd12d17fc2e5a1f9e8c97
129000 616bf48117fc2e5a1f9e8d09
168000 616bf95e17fc2e5a1f9e8d16
166000 616bfc4c17fc2e5a1f9e8d24
236000 616c17b217fc2e5a1f9e8d8e
38700 616c1e9617fc2e5a1f9e8da5
106000 616c2d1317fc2e5a1f9e8dd5
159200 616c316c17fc2e5a1f9e8e05
305800 616c331a17fc2e5a1f9e8e0b
131670 616c3c7c17fc2e5a1f9e8e1f
637030 616c3cbf17fc2e5a1f9e8e22
131670 616c6ab017fc2e5a1f9e8e42
131670 616ca14b17fc2e5a1f9e8ead
97350 616d47d317fc2e5a1f9e8fbe
82000 616d495d17fc2e5a1f9e8fc3
230670 616d560317fc2e5a1f9e8ffc
166350 616d631317fc2e5a1f9e9020
166350 616d707617fc2e5a1f9e907d
113700 616d712e17fc2e5a1f9e9085
113700 616d716017fc2e5a1f9e908a
1430000 616d717c17fc2e5a1f9e908d
298350 616d801417fc2e5a1f9e90f1
156000 616da2